In [40]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse

import locale
import time
from datetime import datetime
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

from tqdm import tqdm

In [41]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf.loc[:,'IP Address'].values[0]
}

### object detail
id: int  
screen_name: @usuario  
tweet_count: int  
description: descripcion usuario 
location: ubicacion en desc  
url: url en desc  
listed_count: listas de las cuales forma parte  
created_at: cuenta creada el  
crawled_at: cuando se escrapeo  
favs_count: int  
followers_count: int  
following_count: int
verified: true/false

In [42]:
#NO DESCOMENTAR!
# userDf = pd.DataFrame(columns=[
#     'id',
#     'screen_name',
#     'complete_name',
#     'tweet_count',
#     'description',
#     'location',
#     'birthdate',
#     'url',
#     'listed_count',
#     'created_at',
#     'crawled_at',
#     'favs_count',
#     'followers_count',
#     'following_count',
#     'verified'
# ])

userDf = pd.read_pickle('../datasets/UsersDataset.pkl')
toScrap = pd.read_pickle('../datasets/targetUsersDataset.pkl')

#saco los que ya escrapie
display(userDf.head(2))
print('Usuarios ya existentes', len(userDf))
print('\n\n')
display(toScrap.head(2))
print('Targets totales', len(toScrap))
toScrap.index = toScrap.index.map(int)
toScrap = toScrap.loc[toScrap.index.difference(userDf.index).values]
print('Target restantes', len(toScrap))

,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
index,,,,,,,,,,,,,,,
2300560809,2300560809,NMoraniB,Nicole Morani Brown,593,Gerencia General de @PuertoBsAs Ministerio de ...,"Buenos Aires, Argentina",False,False,0,2014-01-01 00:00:00,2019-06-29 23:04:21.059135,2377,1630,1510,False
1536923558,1536923558,FabianPereyra87,Fabian Pereyra,2532,,False,False,False,0,2013-06-01 00:00:00,2019-06-29 23:04:21.289518,4307,53,284,False


Usuarios ya existentes 7540





,username
235491971,gsrcaballero
497428913,juannfalopa


Targets totales 15805
Target restantes 8266


In [5]:
def scrap_users(toScrap):
    for username in tqdm(toScrap):
        pageResponse = requests.get('https://twitter.com/'+username, headers=headersChrome, proxies=proxies)
        pageParsed = BeautifulSoup(pageResponse.text, 'html.parser')
        profileNavItems = {
            'tweets': 0,
            'following': 0,
            'followers': 0,
            'favorites': 0,
            'moments': 0,
            'lists': 0
        }
        for itemName in profileNavItems.keys():
            liSearch = pageParsed.findAll(attrs={'class':'ProfileNav-item--'+itemName})
            if(len(liSearch)>0):
                liValue = liSearch[0].findAll(attrs={'class':'ProfileNav-value'})[0]
                if( 'data-count' in  liValue.attrs.keys()):
                    profileNavItems[itemName] = liValue.attrs['data-count']
                else:
                    profileNavItems[itemName] = liValue.text
        completeName = pageParsed.findAll('h1', attrs={'class':'ProfileHeaderCard-name'})[0].findAll('a')[0].text
        completeName = completeName.replace(u'\xa0', u' ')
        completeName = completeName.replace(u'\n', ' ')
        bio = pageParsed.findAll(attrs={'class':'ProfileHeaderCard-bio'})[0].text
        bio = bio.replace(u'\xa0', u' ')
        bio = bio.replace(u'\n', ' ')
        isVerified = False
        if(len(pageParsed.findAll(attrs={'class':'ProfileHeaderCard-name'})[0].findAll(attrs={'class':'Icon--verified'}))>0):
            isVerified = True
        userId = int(pageParsed.findAll(attrs={'class':'ProfileNav'})[0].attrs['data-user-id'])
        profileHeaderItems = {
            'location': False,
            'url': False,
            'joinDate': False,
            'birthdate': False
        }
        for itemName in profileHeaderItems.keys():
            liSearch = pageParsed.findAll(attrs={'class':'ProfileHeaderCard-'+itemName})
            if(len(liSearch)>0):
                if('u-hidden' not in liSearch[0].attrs['class']):
                    liValue = liSearch[0].findAll(attrs={'class':'ProfileHeaderCard-'+itemName+'Text'})[0].text
                    liValue = re.sub(u'\n', ' ', liValue)
                    liValue = liValue.strip(' ')
                    liValue = liValue.replace(u'\xa0', u' ')
                    if(itemName == 'joinDate'):
                        liValue = re.sub(r'Se unió en ', '', liValue)
                        liValue = re.sub(r' de ', ' ', liValue)
                        liValue = datetime.strptime(liValue, '%B %Y')
                        liValue = pd.Timestamp(ts_input=liValue)
                    if(itemName == 'birthdate'):
                        liValue = re.sub(r'Fecha de nacimiento ', '', liValue)
                    profileHeaderItems[itemName] = liValue
        userData = {
            'id': userId,
            'screen_name': username,
            'complete_name': completeName,
            'description': bio,
            'verified': isVerified,
            'crawled_at': pd.Timestamp.now(),
            'location': profileHeaderItems['location'],
            'url': profileHeaderItems['url'],
            'created_at': profileHeaderItems['joinDate'],
            'birthdate': profileHeaderItems['birthdate'],
            'tweet_count': profileNavItems['tweets'],
            'listed_count': profileNavItems['lists'],
            'favs_count': profileNavItems['favorites'],
            'followers_count': profileNavItems['followers'],
            'following_count': profileNavItems['following']
            
        }
        userDf.loc[userData['id']] = userData



In [6]:
# Funcion para Fraccionar el universo a scrapear
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

In [7]:
# Función para paralelizar
fracciones = 8
cuenta_errores = 0
from joblib import Parallel, delayed

def parascrap(arg):
    cuenta_errores = 0
    try:
        scrap_users(arg)
        return result
    except:
        print('Error')
        cuenta_errores += 1

result = Parallel(n_jobs=fracciones, backend="threading")(map(delayed(parascrap),chunkIt(toScrap.screen_name.values,fracciones)))


  0%|          | 0/1976 [00:00<?, ?it/s]





  0%|          | 0/1976 [00:00<?, ?it/s]

  0%|          | 0/1975 [00:00<?, ?it/s]


  0%|          | 0/1976 [00:00<?, ?it/s]



  0%|          | 0/1975 [00:00<?, ?it/s]




  0%|          | 0/1975 [00:00<?, ?it/s]






  0%|          | 1/1976 [00:17<9:36:28, 17.51s/it][A




  0%|          | 1/1976 [00:22<12:09:21, 22.16s/it]



  0%|          | 1/1975 [00:28<15:44:15, 28.70s/it]





  0%|          | 1/1976 [00:29<15:55:06, 29.02s/it]






  0%|          | 1/1976 [00:30<16:24:48, 29.92s/it]


  0%|          | 1/1976 [00:31<17:02:25, 31.06s/it]

  0%|          | 2/1976 [00:46<11:28:31, 20.93s/it]





  0%|          | 2/1976 [00:55<15:26:03, 28.15s/it]




  0%|          | 2/1975 [00:57<13:07:34, 23.95s/it]



  0%|          | 2/1975 [01:01<16:23:04, 29.90s/it]






  0%|          | 2/1976 [01:02<16:43:27, 30.50s/it]

  0%|          | 3/1976 [01:06<11:20:17, 20.69s/it]


  0%|          | 2/1976 [01:08<18:07:19, 33.05s/it]





  0%|    

Error








  3%|▎         | 61/1976 [25:50<16:03:02, 30.17s/it]

  3%|▎         | 59/1975 [26:01<12:53:03, 24.21s/it]






  3%|▎         | 58/1976 [26:05<11:50:25, 22.22s/it]



  3%|▎         | 58/1976 [26:07<16:27:01, 30.88s/it]


  3%|▎         | 52/1976 [26:07<18:38:54, 34.89s/it]




  3%|▎         | 60/1975 [26:08<13:02:16, 24.51s/it]





  3%|▎         | 62/1976 [26:09<14:13:37, 26.76s/it]






  3%|▎         | 59/1976 [26:26<11:45:10, 22.07s/it]


  3%|▎         | 53/1976 [26:27<16:16:50, 30.48s/it]

  3%|▎         | 60/1975 [26:28<13:15:56, 24.94s/it]



  3%|▎         | 63/1975 [26:28<12:00:15, 22.60s/it]





  3%|▎         | 63/1976 [26:29<13:03:19, 24.57s/it]




  3%|▎         | 59/1976 [26:34<16:12:00, 30.42s/it]



  3%|▎         | 64/1975 [26:50<11:50:46, 22.32s/it]

  3%|▎         | 61/1975 [26:50<12:54:42, 24.29s/it]






  3%|▎         | 60/1976 [26:51<12:13:18, 22.96s/it]


  3%|▎         | 54/1976 [26:52<15:17:50, 28.65s/it]




  3%|▎         | 62/1975 [26:52<12:

Error









 10%|█         | 204/1976 [1:27:23<10:07:01, 20.55s/it]




 10%|█         | 199/1975 [1:27:25<10:16:36, 20.83s/it]


  9%|▉         | 187/1976 [1:27:25<10:07:12, 20.36s/it]

 10%|█         | 199/1976 [1:27:26<9:38:55, 19.55s/it] 



 11%|█         | 209/1975 [1:27:26<10:13:23, 20.84s/it]






 10%|█         | 205/1976 [1:27:38<9:13:33, 18.75s/it] 

 10%|█         | 200/1976 [1:27:40<8:54:06, 18.04s/it]



 11%|█         | 210/1975 [1:27:41<9:26:01, 19.24s/it] 


 10%|▉         | 188/1976 [1:27:54<11:14:06, 22.62s/it]

 11%|█         | 209/1975 [1:27:55<8:32:07, 17.40s/it]






 10%|█         | 206/1976 [1:27:58<9:33:05, 19.43s/it]




 10%|█         | 200/1975 [1:27:59<12:27:36, 25.27s/it]



 10%|█         | 201/1976 [1:28:06<10:02:48, 20.38s/it][A


 10%|▉         | 189/1976 [1:28:09<10:16:50, 20.71s/it]

 11%|█         | 210/1975 [1:28:10<8:07:14, 16.56s/it]



 11%|█         | 212/1975 [1:28:11<8:13:17, 16.79s/it]






 10%|█         | 207/1976 [1:28:12<8:44:56, 17.80s/i

...

Error





 20%|█▉        | 393/1976 [2:26:53<4:47:24, 10.89s/it]

 22%|██▏       | 433/1975 [2:26:53<4:37:45, 10.81s/it]

 22%|██▏       | 434/1975 [2:27:01<4:14:00,  9.89s/it]


 20%|█▉        | 394/1976 [2:27:01<4:26:24, 10.10s/it]


 20%|█▉        | 395/1976 [2:27:10<4:16:29,  9.73s/it]

 22%|██▏       | 435/1975 [2:27:11<4:13:43,  9.89s/it]


 20%|██        | 396/1976 [2:27:17<3:54:50,  8.92s/it]

 22%|██▏       | 436/1975 [2:27:23<4:27:45, 10.44s/it]


 20%|██        | 397/1976 [2:27:24<3:38:01,  8.28s/it]

 22%|██▏       | 437/1975 [2:27:28<3:50:14,  8.98s/it]


 20%|██        | 398/1976 [2:27:29<3:15:08,  7.42s/it]

 22%|██▏       | 438/1975 [2:27:33<3:18:24,  7.75s/it]


 20%|██        | 399/1976 [2:27:34<2:50:56,  6.50s/it]

 22%|██▏       | 439/1975 [2:27:38<3:00:40,  7.06s/it]


 20%|██        | 400/1976 [2:27:39<2:43:34,  6.23s/it]

 22%|██▏       | 440/1975 [2:27:45<2:58:24,  6.97s/it]


 20%|██        | 401/1976 [2:27:47<2:51:05,  6.52s/it]

 22%|██▏       | 441/1975 [2:27:50<2:

Error





 54%|█████▎    | 1060/1976 [3:42:58<1:01:20,  4.02s/it]


 54%|█████▎    | 1061/1976 [3:43:01<54:28,  3.57s/it]  


 54%|█████▎    | 1062/1976 [3:43:03<49:49,  3.27s/it]


 54%|█████▍    | 1063/1976 [3:43:06<48:46,  3.20s/it]


 54%|█████▍    | 1064/1976 [3:43:11<57:00,  3.75s/it]


 54%|█████▍    | 1065/1976 [3:43:14<51:19,  3.38s/it]


 54%|█████▍    | 1066/1976 [3:43:16<46:37,  3.07s/it]


 54%|█████▍    | 1067/1976 [3:43:19<43:05,  2.84s/it]


 54%|█████▍    | 1068/1976 [3:43:21<41:20,  2.73s/it]


 54%|█████▍    | 1069/1976 [3:43:24<40:58,  2.71s/it]


 54%|█████▍    | 1070/1976 [3:43:26<40:27,  2.68s/it]


 54%|█████▍    | 1071/1976 [3:43:29<38:42,  2.57s/it]


 54%|█████▍    | 1072/1976 [3:43:32<41:27,  2.75s/it]


 54%|█████▍    | 1073/1976 [3:43:34<39:13,  2.61s/it]


 54%|█████▍    | 1074/1976 [3:43:36<38:13,  2.54s/it]


 54%|█████▍    | 1075/1976 [3:43:40<41:02,  2.73s/it]


 54%|█████▍    | 1076/1976 [3:43:44<46:54,  3.13s/it]


 55%|█████▍    | 1077/1976 [3:43:46<44:18

Error


In [11]:
print('El scrapeo fallo para ',cuenta_errores, ' usuarios')
userDf.info()

El scrapeo fallo para  0  usuarios
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4496 entries, 868668614764875777 to 1116978217477443584
Data columns (total 15 columns):
id                 4496 non-null object
screen_name        4496 non-null object
complete_name      4496 non-null object
tweet_count        4496 non-null object
description        4496 non-null object
location           4496 non-null object
birthdate          4496 non-null object
url                4496 non-null object
listed_count       4496 non-null object
created_at         4496 non-null datetime64[ns]
crawled_at         4496 non-null datetime64[ns]
favs_count         4496 non-null object
followers_count    4496 non-null object
following_count    4496 non-null object
verified           4496 non-null object
dtypes: datetime64[ns](2), object(13)
memory usage: 562.0+ KB


In [9]:
# Descargar a un archivo
userDf.to_pickle('../datasets/UsersDataset.pkl')